## Load data

In [8]:
sys.path.append("..")

from src.utils import *

import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

account_df = read_to_df("account.csv")
card_test_df = read_to_df("card_test.csv")
card_train_df = read_to_df("card_train.csv")
client_df = read_to_df("client.csv")
disp_df = read_to_df("disp.csv")
district_df = read_to_df("district.csv")
loan_test_df = read_to_df("loan_test.csv")
loan_train_df = read_to_df("loan_train.csv")
trans_test_df = read_to_df("trans_test.csv")
trans_train_df = read_to_df("trans_train.csv")

C:\Users\skelo\AppData\Local\Temp/ipykernel_9884/3219962980.py:14: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  trans_train_df = read_to_df("trans_train.csv")


## Process account data

In [13]:
account_df.head()

,account_id,district_id,frequency,date
0,576,55,monthly issuance,930101
1,3818,74,monthly issuance,930101
2,704,55,monthly issuance,930101
3,2378,16,monthly issuance,930101
4,2632,24,monthly issuance,930102


## Process client data

In [15]:
client_df["sex"] = client_df["birth_number"].apply(lambda x: 0 if int(str(x)[2:4]) > 50 else 1) 
client_df["age"] = client_df["birth_number"].apply(lambda x: calculate_age(read_date(x)))
client_df.drop("birth_number", inplace=True, axis=1)

client_df.head()

,client_id,district_id,sex,age
0,1,18,0,28
1,2,1,1,53
2,3,1,0,58
3,4,5,1,42
4,5,5,0,38


In [16]:
client_district_df = client_df.join(district_df)
client_district_df.head()

,client_id,district_id,sex,age,code,name,region,no. of inhabitants,no. of municipalities with inhabitants < 499,no. of municipalities with inhabitants 500-1999,no. of municipalities with inhabitants 2000-9999,no. of municipalities with inhabitants >10000,no. of cities,ratio of urban inhabitants,average salary,unemploymant rate '95,unemploymant rate '96,no. of enterpreneurs per 1000 inhabitants,no. of commited crimes '95,no. of commited crimes '96
0,1,18,0,28,1.0,Hl.m. Praha,Prague,1204953.0,0.0,0.0,0.0,1.0,1.0,100.0,12541.0,0.29,0.43,167.0,85677,99107.0
1,2,1,1,53,2.0,Benesov,central Bohemia,88884.0,80.0,26.0,6.0,2.0,5.0,46.7,8507.0,1.67,1.85,132.0,2159,2674.0
2,3,1,0,58,3.0,Beroun,central Bohemia,75232.0,55.0,26.0,4.0,1.0,5.0,41.7,8980.0,1.95,2.21,111.0,2824,2813.0
3,4,5,1,42,4.0,Kladno,central Bohemia,149893.0,63.0,29.0,6.0,2.0,6.0,67.4,9753.0,4.64,5.05,109.0,5244,5892.0
4,5,5,0,38,5.0,Kolin,central Bohemia,95616.0,65.0,30.0,4.0,1.0,6.0,51.4,9307.0,3.85,4.43,118.0,2616,3040.0


In [9]:
joined_loan_test_df = loan_test_df.merge(card_test_df.merge(disp_df, on="disp_id", how="left"), on="account_id", how="left")
joined_loan_test_df.rename(columns={"type_x": "card_type"}, inplace=True)
joined_loan_test_df["card_type"].replace({"classic": 1, "junior": 2, "gold": 3}, inplace=True) 

joined_loan_test_df.head()
loan_test_df.head()

,loan_id,account_id,date,amount,duration,payments,status
0,5895,4473,970103,93960,60,1566,NaN
1,7122,10365,970104,260640,36,7240,NaN
2,6173,5724,970108,232560,48,4845,NaN
3,6142,5591,970121,221880,60,3698,NaN
4,5358,2018,970121,38520,12,3210,NaN


In [10]:
joined_loan_train_df = loan_train_df.merge(card_train_df.merge(disp_df, on="disp_id"), on="account_id")
joined_loan_train_df.rename(columns={"type_x": "card_type"}, inplace=True)

# Replace text with numeric data
joined_loan_train_df["card_type"].replace({"classic": 1, "junior": 2, "gold": 3}, inplace=True) 
joined_loan_train_df["type_y"].replace({"OWNER": 1, "USER": 2}, inplace=True) 

joined_loan_train_df.head(50)
# joined_loan_train_df.count() #Doesn't count NA values

,loan_id,account_id,date,amount,duration,payments,status,card_id,disp_id,card_type,issued,client_id,type_y
0,6577,7753,940311,51696,24,2154,1,1005,9285,1,931107,9593,1
1,5595,3050,940902,82896,12,6908,1,577,3687,1,940215,3687,1
2,7243,11042,951204,217152,36,6032,1,1225,13268,1,951122,13576,1
3,5900,4503,960101,135360,48,2820,1,783,5433,2,951024,5433,1
4,6550,7614,960518,320592,48,6679,1,989,9119,1,960221,9427,1
5,5311,1766,960615,30060,60,501,1,337,2141,1,960101,2141,1
6,6992,9790,960727,133968,48,2791,1,1134,11742,1,960531,12050,1
7,7305,11359,960806,54024,12,4502,1,1247,13660,1,950613,13968,1
8,5088,544,960822,91152,24,3798,1,119,655,1,960808,655,1
9,7154,10520,961205,417600,60,6960,1,1187,12635,3,960717,12943,1


In [11]:
loan_train_df.head()

,loan_id,account_id,date,amount,duration,payments,status
0,5314,1787,930705,96396,12,8033,-1
1,5316,1801,930711,165960,36,4610,1
2,6863,9188,930728,127080,60,2118,1
3,5325,1843,930803,105804,36,2939,1
4,7240,11013,930906,274740,60,4579,1


## Train model and predict

In [16]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

X = loan_train_df.drop(["loan_id", "account_id", "status"], axis=1)
y = loan_train_df["status"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

# Create Decision Tree classifier object
classifier = DecisionTreeClassifier()

# Train Decision Tree classifier
classifier = classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

## Metrics


In [17]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.7926829268292683

## Save results

In [18]:
X_result = loan_test_df.drop(["loan_id", "account_id", "status"], axis=1)

y_result = classifier.predict(X_result)

result = pd.DataFrame({"Id": loan_test_df["loan_id"], "Predicted": y_result})
result.to_csv("../out/result.csv", index=False)